# Introduction to Financial Python
## Tutorial 8 - Confidence Interval and Hypothesis Testing

### Intervalo de confianza

#### Muestra de error
Utilicemos la rentabilidad diaria de Apple desde agosto del 2010 hasta la actualidad como población.

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

qb = QuantBook()
appl = qb.AddEquity("AAPL").Symbol

start_date = datetime(2010, 8, 1, 0, 0, 0)
end_date = qb.Time
appl_table = qb.History(appl, start_date, end_date, Resolution.Daily)
appl_total = appl_table[['open','close']]

# Calcular los rendimientos de los registros
appl_log_return = np.log(appl_total.close).diff().dropna()

print('Media de la población:', np.mean(appl_log_return))
print('Desviación estándar de la población:',np.std(appl_log_return))

Media de la población: 0.001011261840353075
Desviación estándar de la población: 0.017848654797024178


Ahora vamos a comprobar la muestra de los últimos 10 días y la muestra de los últimos 1000 días.

In [9]:
print('Devolución de muestras en 10 días:', np.mean(appl_log_return.tail(10)))
print('Desviación estándar de muestras en 10 días::', np.std(appl_log_return.tail(10)))
print('Devolución de muestras en 1000 días:', np.mean(appl_log_return.tail(1000)))
print('Desviación estándar de muestras en 1000 días::', np.std(appl_log_return.tail(1000)))

Devolución de muestras en 10 días: 0.00014861297319077594
Desviación estándar de muestras en 10 días:: 0.01809897239385377
Devolución de muestras en 1000 días: 0.0012654006104160236
Desviación estándar de muestras en 1000 días:: 0.020515081561995697


Como esperábamos, las dos muestras tienen medias y varianzas diferentes.

#### Intervalo de confianza
Para estimar el rango de la media poblacional, definimos el error estándar de la media como sigue: <br>
$ SE = \frac{\sigma}{\sqrt{n}}$, donde $\sigma$ es la desviación estándar de la muestra y n es el tamaño de la muestra. <br>
Generalmente, si queremos estimar un intervalo de la población para que el 95% de las veces el intervalo contenga la media poblacional, el intervalo se calcula como: <br>
$ (\mu -1.96*SE, \mu +1.96*SE)$, donde $\mu$ es la media de la muestra y SE es el error estándar. <br>
Solemos utilizar 1,96 para calcular un intervalo de confianza del 95% porque suponemos que la media de la muestra sigue una distribución normal.

In [10]:
bottom_1 = np.mean(appl_log_return.tail(10))-1.96*np.std(appl_log_return.tail(10))/(np.sqrt(len((appl_log_return.tail(10)))))
upper_1 = np.mean(appl_log_return.tail(10))+1.96*np.std(appl_log_return.tail(10))/(np.sqrt(len((appl_log_return.tail(10)))))
bottom_2 = np.mean(appl_log_return.tail(1000))-1.96*np.std(appl_log_return.tail(1000))/(np.sqrt(len((appl_log_return.tail(1000)))))
upper_2 = np.mean(appl_log_return.tail(1000))+1.96*np.std(appl_log_return.tail(1000))/(np.sqrt(len((appl_log_return.tail(1000)))))

print('10 días, invervalo de confianza del 95%:', (bottom_1,upper_1))
print('1000 días, invervalo de confianza del 95%:', (bottom_2,upper_2))

10 días, invervalo de confianza del 95%: (-0.011069246337134471, 0.011366472283516022)
1000 días, invervalo de confianza del 95%: (-6.137318336588236e-06, 0.0025369385391686354)


#### Intervalo de confianza de la distribución normal
La distribución normal se utiliza con tanta frecuencia que deberíamos ser capaces de recordar algunos valores críticos de la misma. En concreto, solemos utilizar el 90%, el 95% y el 99% como nivel de confianza de un intervalo de confianza. Los valores críticos para estos tres niveles de confianza son 1,64, 1,96 y 2,32 respectivamente.

#### Teoría del límite central
Como hemos dicho, si utilizamos la muestra para estimar el intervalo de confianza de la población, el intervalo de confianza del 95% es: <br>
$ (\mu -1.96*SE, \mu +1.96*SE)$ <br>
Este teorema nos dice que, dado un tamaño de muestra suficientemente grande de una población con un nivel finito de varianza, la media de todas las muestras de la misma población será aproximadamente igual a la media de la población, y las medias de las muestras tendrán una distribución aproximadamente normal. 

### Pruebas de hipótesis

La prueba de hipótesis es esencialmente una prueba de inferencia basada en una muestra.<br>
Supongamos que no conocemos la media de esta población. Supongo que la media de esta población es 0. ¿Es correcta mi suposición? Tengo que comprobar esta hipótesis con mi muestra. Empecemos por observar nuestra muestra: 

In [11]:
mean_1000 = np.mean(appl_log_return.tail(1000))
std_1000 = np.std(appl_log_return.tail(1000))
mean_10 = np.mean(appl_log_return.tail(10))
std_10 = np.std(appl_log_return.tail(10))
s = pd.Series([mean_10,std_10,mean_1000,std_1000],index = ['media_10', 'std_10','media_1000','std_1000'])
print(s)

media_10      0.000149
std_10        0.018099
media_1000    0.001265
std_1000      0.020515
dtype: float64


Ahora sabemos cómo calcular el intervalo de confianza. Si tuviera razón, es decir, la media de la población es 0, entonces el intervalo de confianza del 90% de la muestra con 1000 observaciones debería ser: 

In [12]:
bottom = 0 - 1.64*std_1000/np.sqrt(1000)
upper = 0 + 1.64*std_1000/np.sqrt(1000)
print((bottom, upper))

(-0.001063939899568512, 0.001063939899568512)


Nuestra media de la muestra está dentro del intervalo de confianza del 90%, así que podemos decir que la hipótesis es correcta.

En general, tenemos la hipótesis nula $ H_0 $ y la hipótesis alternativa. Suelen tener las siguientes formas: <br>
$ H_0 : \bar{\mu} = 0 $ <br>
$ H_0 : \bar{\mu} \not= 0 $ <br>
Si el valor analizado está fuera del intervalo de confianza, rechazamos la hipótesis nula o aceptamos la hipótesis alternativa. También podemos invertir el proceso para calcular el valor crítico, o puntuación Z. La puntuación Z se define como: <br>
$ Z = \frac{x-\mu}{\frac{\sigma}{\sqrt{n}}} $

In [14]:
print(np.sqrt(1000)*(mean_1000 - 0)/std_1000)

1.9505396892474032


En nuestro ejemplo, la puntuación z es 1.9505. Podemos conocer la anchura es el intervalo de confianza refiriéndonos a una tabla de distribución normal. Por supuesto, podemos hacer esto en Python:

In [15]:
import scipy.stats as st
print((1 - st.norm.cdf(1.9505)))

0.02555827669133459


Este número calculado se llama valor p. <br>
Ahora vamos a probar la hipótesis de que la media de la población = 0 de nuevo con una muestra grande, que tiene 1200 observaciones:

In [16]:
mean_1200 = np.mean(appl_log_return.tail(1200))
std_1200 = np.std(appl_log_return.tail(1200))
z_score = np.sqrt(1200)*(mean_1200 - 0)/std_1200
print('z-score = ',z_score)
p_value = (1 - st.norm.cdf(z_score))
print('p_value = ',p_value)

z-score =  2.537215239096944
p_value =  0.005586911551146101
